In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [18]:
# Load the CIFAR10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
# Define the CNN model with leaky ReLu
class CNNLeakyReLu(nn.Module):
    def __init__(self):
        super(CNNLeakyReLu, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(in_features=32 * 8 * 8, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)
        self.leakyrelu = nn.LeakyReLU()

    def forward(self, x):
        x = self.leakyrelu(self.conv1(x))
        x = self.pool(x)
        x = self.leakyrelu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.leakyrelu(self.fc1(x))
        x = self.fc2(x)
        return x

class CNNTanh(nn.Module):
    def __init__(self):
        super(CNNTanh, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(in_features=32 * 8 * 8, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.tanh(self.conv1(x))
        x = self.pool(x)
        x = self.tanh(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.tanh(self.fc1(x))
        x = self.fc2(x)
        return x


In [21]:
# Define the dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

criterion = nn.CrossEntropyLoss()

def evaluate_model(model, optimizer):
    writer = SummaryWriter()

    # Train the model
    for epoch in range(10):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                writer.add_scalar('training loss', running_loss / 100, epoch * len(train_loader) + i)
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0
                
    writer.close()

    # Evaluate the model on the test set
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

leaky_relu_model = CNNLeakyReLu()
tanh_model = CNNTanh()

res1 = evaluate_model(leaky_relu_model, optim.SGD(leaky_relu_model.parameters(), lr=0.0001))
res2 = evaluate_model(leaky_relu_model, optim.Adam(leaky_relu_model.parameters(), lr=0.0001))
res3 = evaluate_model(tanh_model, optim.Adam(leaky_relu_model.parameters(), lr=0.0001))

print(f"Accuracy of Leaky ReLu with SGD: {res1}%")
print(f"Accuracy of Leaky ReLu with Adam: {res2}%")
print(f"Accuracy of Tanh with Adam: {res3}%")

[1,   100] loss: 2.305
[1,   200] loss: 2.304
[1,   300] loss: 2.302
[1,   400] loss: 2.305
[1,   500] loss: 2.304
[1,   600] loss: 2.303
[1,   700] loss: 2.304
[2,   100] loss: 2.302
[2,   200] loss: 2.303
[2,   300] loss: 2.302
[2,   400] loss: 2.301
[2,   500] loss: 2.302
[2,   600] loss: 2.301
[2,   700] loss: 2.300
[3,   100] loss: 2.300
[3,   200] loss: 2.300
[3,   300] loss: 2.299
[3,   400] loss: 2.300
[3,   500] loss: 2.301
[3,   600] loss: 2.300
[3,   700] loss: 2.300
[4,   100] loss: 2.298
[4,   200] loss: 2.298
[4,   300] loss: 2.299
[4,   400] loss: 2.297
[4,   500] loss: 2.299
[4,   600] loss: 2.299
[4,   700] loss: 2.298
[5,   100] loss: 2.297
[5,   200] loss: 2.297
[5,   300] loss: 2.296
[5,   400] loss: 2.298
[5,   500] loss: 2.297
[5,   600] loss: 2.296
[5,   700] loss: 2.297
[6,   100] loss: 2.295
[6,   200] loss: 2.296
[6,   300] loss: 2.296
[6,   400] loss: 2.295
[6,   500] loss: 2.294
[6,   600] loss: 2.296
[6,   700] loss: 2.294
[7,   100] loss: 2.294
[7,   200] 

![alt text](image.png "Title")